In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
import warnings
import optuna
import scipy.stats as st
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings('ignore')

In [ ]:
test = pd.read_csv(r"C:\Users\jiahe\OneDrive\Desktop\kaggle\test.csv")
train = pd.read_csv(r"C:\Users\jiahe\OneDrive\Desktop\kaggle\train.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

In [ ]:
train.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(train['accident_risk'], bins=30, kde=True, color='teal')
plt.title("Distribution of Accident Risk")
plt.xlabel("accident_risk")
plt.ylabel("Frequency")
plt.show()

In [ ]:
train_cat = train.copy()
test_cat = test.copy()

cat_cols = ['road_type', 'lighting', 'weather', 'time_of_day']

for col in cat_cols:
    categories = pd.Categorical(pd.concat([train_cat[col], test_cat[col]], axis=0)).categories
    train_cat[col] = pd.Categorical(train_cat[col], categories=categories)
    test_cat[col] = pd.Categorical(test_cat[col], categories=categories)

In [ ]:
# List of categorical columns to target encode
target_encode_cols = ['road_type', 'lighting', 'weather', 'time_of_day']

# Initialize KFold for target encoding
kf = KFold(n_splits=5, shuffle=True, random_state=42)
encoded_features = pd.DataFrame(index=train.index)

for col in target_encode_cols:
    print(f"Target encoding: {col}")
    oof = pd.Series(np.nan, index=train.index)

    # Create out-of-fold mean for each category
    for train_idx, val_idx in kf.split(train):
        means = train.iloc[train_idx].groupby(col)['accident_risk'].mean()
        oof.iloc[val_idx] = train.iloc[val_idx][col].astype(str).map(means)


    encoded_features[col + '_te'] = oof

# Add encoded features to train
train = pd.concat([train, encoded_features], axis=1)

# Apply same encoding (using full-train mean) to test
for col in target_encode_cols:
    global_mean = train.groupby(col)['accident_risk'].mean()
    test[col + '_te'] = test[col].astype(str).map(global_mean)

In [ ]:
# =====================================================
# === Define baseline analytic function and clipper ===
# =====================================================
def f_base(X):
    """Simple analytic approximation of accident risk."""
    return (
        0.3 * X["curvature"]
        + 0.2 * (X["lighting"] == "night").astype(int)
        + 0.1 * (X["weather"] != "clear").astype(int)
        + 0.2 * (X["speed_limit"] >= 60).astype(int)
        + 0.1 * (X["num_reported_accidents"] > 2).astype(int)
    )

def clip(f, sigma=0.05):
    """Smoothly clip predictions into [0,1] using truncated normal expectation."""
    def clip_f(X):
        mu = f(X)
        a, b = -mu/sigma, (1 - mu)/sigma
        Phi_a, Phi_b = st.norm.cdf(a), st.norm.cdf(b)
        phi_a, phi_b = st.norm.pdf(a), st.norm.pdf(b)
        return mu*(Phi_b - Phi_a) + sigma*(phi_a - phi_b) + 1 - Phi_b
    return clip_f

# Apply baseline to train and test
train["y_base"] = clip(f_base)(train)
test["y_base"] = clip(f_base)(test)

# Compute residual target
train["y_resid"] = train["accident_risk"] - train["y_base"]

In [ ]:
X = train_cat.drop(columns=['id', 'accident_risk'])
y = train['y_resid']
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
importances = pd.DataFrame()
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Fold {fold+1}")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model_fs = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.03,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )

    model_fs.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[early_stopping(100)]
    )

    fold_importance = pd.DataFrame({
        'feature': X.columns,
        'gain': model_fs.booster_.feature_importance(importance_type='gain'),
        'fold': fold + 1
    })
    importances = pd.concat([importances, fold_importance], axis=0)

# Compute feature importance
importance_df = (
    importances.groupby('feature')['gain']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# Filter weak features
threshold = importance_df['gain'].median() * 0.1
selected_features = importance_df.query('gain > @threshold')['feature'].tolist()

print(f"✅ Selected {len(selected_features)} / {len(X.columns)} features after multi-fold averaging.")

# Reduce the dataset to selected features
X = X[selected_features]
test_X = test_cat[selected_features]

In [ ]:
def objective(trial):
    params = {
        "n_estimators": 8000,
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 200),
        "random_state": 42,
        "n_jobs": -1,
        "verbosity": -1
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse",
            callbacks=[optuna.integration.LightGBMPruningCallback(trial, "rmse")]
        )
        preds = model.predict(X_val)
        mse = mean_squared_error(y_val, preds)
        rmse = np.sqrt(mse)
        rmse_scores.append(rmse)
        
    return np.mean(rmse_scores)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("Best RMSE:", study.best_value)
print("Best Params:", study.best_params)

best_params = study.best_params

In [ ]:
# Set up 8-Fold CV
kf = KFold(n_splits=8, shuffle=True, random_state=42)
rmse_scores = []
test_X = test_cat.drop(columns=['id'])
test_preds_all = np.zeros(len(test_X))

In [ ]:
# store out-of-fold preds for later alpha tuning
oof_lgb = np.zeros(len(X))
oof_y = y.values.copy()
oof_rf = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n===== Fold {fold + 1} / {kf.n_splits} =====")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # Align columns
    X_val = X_val[X_train.columns]
    test_X = test_X[X_train.columns]

    # === LightGBM model (tuned) ===
    model_lgb = LGBMRegressor(
        **study.best_params,
        n_estimators=12000,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )

    model_lgb.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[early_stopping(200), log_evaluation(0)]
    )
    
    preds_lgb = model_lgb.predict(X_val)
    oof_lgb[val_idx] = preds_lgb

    # === Random Forest model ===
    # === Prepare numeric data for Random Forest ===
    X_train_rf = pd.get_dummies(X_train, drop_first=True)
    X_val_rf   = pd.get_dummies(X_val, drop_first=True)

    # Align columns (since get_dummies may create mismatched columns)
    X_train_rf, X_val_rf = X_train_rf.align(X_val_rf, join='left', axis=1, fill_value=0)

    model_rf = RandomForestRegressor(
        n_estimators=500,
        max_depth=15,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    model_rf.fit(X_train_rf, y_train)
    preds_rf = model_rf.predict(X_val_rf)
    oof_rf[val_idx] = preds_rf

    # temporary equal blend for monitoring
    val_preds = 0.5 * preds_lgb + 0.5 * preds_rf

    rmse = np.sqrt(mean_squared_error(y_val, val_preds))
    rmse_scores.append(rmse)
    print(f"Fold {fold + 1} RMSE: {rmse:.5f}")

    # === Predict on test and accumulate (equal blend for now) ===
    test_X_rf = pd.get_dummies(test_X, drop_first=True)
    test_X_rf = test_X_rf.reindex(columns=X_train_rf.columns, fill_value=0)

    test_pred_lgb = model_lgb.predict(test_X)
    test_pred_rf = model_rf.predict(test_X_rf)
    test_preds_all += (0.5 * test_pred_lgb + 0.5 * test_pred_rf) / kf.n_splits

# Display preliminary CV RMSE
print("\n============================")
print(f"Average RMSE (equal 0.5 blend): {np.mean(rmse_scores):.5f}")
print("============================")

In [ ]:
overall_corr = np.corrcoef(oof_lgb, oof_rf)[0, 1]
print(f"\nOverall OOF correlation between LGBM and RF: {overall_corr:.4f}")

plt.figure(figsize=(6, 5))
sns.kdeplot(x=oof_lgb, y=oof_rf, fill=True, cmap='viridis')
plt.title(f'OOF Prediction Correlation: {overall_corr:.4f}')
plt.xlabel('LGBM OOF Predictions')
plt.ylabel('RF OOF Predictions')
plt.tight_layout()
plt.show()

In [ ]:
# =====================================================
# === Step 2: Tune alpha weight between LGBM & RF ===
# =====================================================
def objective_alpha(trial):
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    blend = alpha * oof_lgb + (1 - alpha) * oof_rf
    return np.sqrt(mean_squared_error(oof_y, blend))

study_alpha = optuna.create_study(direction="minimize")
study_alpha.optimize(objective_alpha, n_trials=40)
best_alpha = study_alpha.best_params["alpha"]
print(f"✅ Best alpha for blending: {best_alpha:.4f}")

In [ ]:
# =====================================================
# === Step 3: Final blending using best alpha ===
# =====================================================
# recompute CV RMSE using best alpha
final_oof = best_alpha * oof_lgb + (1 - best_alpha) * oof_rf
final_rmse = np.sqrt(mean_squared_error(oof_y, final_oof))
print(f"Final blended OOF RMSE: {final_rmse:.5f}")

# re-blend test predictions using alpha
test_pred_final = best_alpha * test_pred_lgb + (1 - best_alpha) * test_pred_rf

In [ ]:
# =====================================================
# === Optional Step: Ridge regression meta-blender ===
# =====================================================
from sklearn.linear_model import RidgeCV

print("\n--- Ridge Regression Meta-Blender ---")

meta_X = np.vstack([oof_lgb, oof_rf]).T
meta_y = oof_y

ridge = RidgeCV(alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10])
ridge.fit(meta_X, meta_y)
ridge_oof = ridge.predict(meta_X)
ridge_rmse = np.sqrt(mean_squared_error(meta_y, ridge_oof))
print(f"Ridge-blend OOF RMSE: {ridge_rmse:.5f}")
print("Ridge coefficients (LGBM, RF):", ridge.coef_)

# Predict on test set using same meta model
test_meta_X = np.vstack([test_pred_lgb, test_pred_rf]).T
test_pred_ridge = ridge.predict(test_meta_X)

# Optionally compare with Optuna alpha blend
print(f"Optuna alpha-blend OOF RMSE: {final_rmse:.5f}")
if ridge_rmse < final_rmse:
    print("✅ Ridge performs slightly better; using Ridge predictions.")
    test_pred_final = test_pred_ridge
else:
    print("ℹ️ Keeping Optuna alpha blend as final.")

In [ ]:
# =====================================================
# === Step 5: Save OOF and test predictions for stacking ===
# =====================================================

# Save OOF predictions for meta-learning later
oof_df = pd.DataFrame({
    'id': train_cat['id'],
    'oof_lgb': oof_lgb,
    'oof_rf': oof_rf,
    'oof_blend': final_oof,   # blended or ridge version
    'y_true': oof_y
})
oof_df.to_csv('oof_lgb_rf.csv', index=False)
print("✅ Saved OOF predictions to oof_lgb_rf.csv")

# Save test predictions (for later external blending)
test_stack_df = pd.DataFrame({
    'id': test_cat['id'],
    'test_lgb': test_pred_lgb,
    'test_rf': test_pred_rf,
    'test_blend': test_pred_final
})
test_stack_df.to_csv('test_lgb_rf.csv', index=False)
print("✅ Saved test predictions to test_lgb_rf.csv")

In [ ]:
# =====================================================
# === Step 4: Save final blended predictions ===
# =====================================================

# Combine optimized test predictions with baseline offset
final_preds = test_pred_final + test["y_base"]

submission = pd.DataFrame({
    'id': test_cat['id'],
    'accident_risk': final_preds
})

submission.to_csv('submission_rfblend.csv', index=False)
print("✅ submission_rfblend.csv ready for Kaggle upload!")